# 16장 도구변수 추정

`statsmodels` 패키지의 경우 도구변수 추정이 복잡하고, `linearmodels` 패키지가 도구변수 추정에 간편한 인터페이스를 제공하나 `statsmodels`와의 통일성이 없다. 특히 `linearmodels`의 formula에서는 모형이 절편을 포함하면 반드시 `1`을 써 주어야 한다. 대부분의 모형에 절편이 포함되고 많은 사용자들이 절편에 관심을 갖지 않음을 고려하면 이는 상당히 비효율적이고 코딩 오류를 불러일으키는 접근방식이다. 또한 `linearmodels`의 default 표준오차는 이분산에 견고한 표준오차라는 점에도 주의하여야 한다.

인터페이스에 통일성이 없다는 것은 매우 불편하고 혼란을 야기한다. 공부할 목적이 아니면 도구변수 추정(또는 『계량경제학강의』의 다른 부분)에 파이썬(python)을 사용하는 것은 추천하지 않는다. (패키지 코드를 수정하는 것은 본 실습의 목적이 아니다.)

## OLS

In [1]:
import pandas as pd
import statsmodels.formula.api as smf

Ivdata = pd.read_csv('csv/loedata/Ivdata.csv')
ols = smf.ols('y~x1+x2', data=Ivdata).fit()
ols.summary(slim=True)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.708
Model:                            OLS   Adj. R-squared:                  0.702
No. Observations:                 100   F-statistic:                     117.4
Covariance Type:            nonrobust   Prob (F-statistic):           1.24e-26
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.3650      0.820     -0.445      0.657      -1.992       1.262
x1             0.4831      0.057      8.458      0.000       0.370       0.597
x2             0.9448      0.064     14.677      0.000       0.817       1.073
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## First-stage regression

In [2]:
stage1 = smf.ols('x2~x1+z2a', data=Ivdata).fit()
stage1.summary(slim=True)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     x2   R-squared:                       0.295
Model:                            OLS   Adj. R-squared:                  0.281
No. Observations:                 100   F-statistic:                     20.33
Covariance Type:            nonrobust   Prob (F-statistic):           4.22e-08
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      6.3374      0.936      6.769      0.000       4.479       8.196
x1            -0.1319      0.079     -1.669      0.098      -0.289       0.025
z2a           21.8401      4.044      5.401      0.000      13.814      29.866
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Second-stage regression

In [3]:
Ivdata['x2hat'] = stage1.fittedvalues
stage2 = smf.ols('y~x1+x2hat', data=Ivdata).fit()
stage2.summary(slim=True)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.138
Model:                            OLS   Adj. R-squared:                  0.120
No. Observations:                 100   F-statistic:                     7.748
Covariance Type:            nonrobust   Prob (F-statistic):           0.000756
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.8827      2.250      0.837      0.405      -2.583       6.348
x1             0.4169      0.111      3.760      0.000       0.197       0.637
x2hat          0.6867      0.230      2.986      0.004       0.230       1.143
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

위 Second-stage regression에 보고되는 표준오차들에는 오류가 있다. 이를 보정해야 하는데, 완전수동 분산 추정이나 반자동 분산 추정은 건너뛰고 완전자동 도구변수 추정으로 넘어간다.

## 완전자동 도구변수 추정

In [4]:
# pip install linearmodels
from linearmodels import IV2SLS

ivfm = 'y~1+x1+[x2~z2a]' # need constant (1) explicitly
ivmodel = IV2SLS.from_formula(ivfm, data=Ivdata)
tsls = ivmodel.fit(cov_type='unadjusted')
tsls

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                      y   R-squared:                      0.6592
Estimator:                    IV-2SLS   Adj. R-squared:                 0.6522
No. Observations:                 100   F-statistic:                    40.418
Date:                Mon, Jun 24 2024   P-value (F-stat)                0.0000
Time:                        08:43:16   Distribution:                  chi2(2)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      1.8827     1.3932     1.3513     0.1766     -0.8479      4.6132
x1             0.4169     0.0687     6.0725     0.0000      0.2824      0.5515
x2             0.6867     0.1424     4.8229     0.0000      0.4076      0.9657
==============================================================================

Endogenous: x2
Instruments: z2a
Unadjusted Covariance (Homoskedastic)
Debiased: False
IVResults, id: 0x12a633710

표준오차가 책과 약간 다른데, 오차분산을 추정할 때 $n-k-1$이 아니라 $n$으로 나누었기 때문이다. $n-k-1$로 나누는 옵션은 보이지 않는다(`help(IV2SLS.fit)` 참고). HC0 유형 표준오차는 다음과 같이 구한다.

In [5]:
tslsh = ivmodel.fit(cov_type='robust')
tslsh

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                      y   R-squared:                      0.6592
Estimator:                    IV-2SLS   Adj. R-squared:                 0.6522
No. Observations:                 100   F-statistic:                    31.850
Date:                Mon, Jun 24 2024   P-value (F-stat)                0.0000
Time:                        08:43:16   Distribution:                  chi2(2)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      1.8827     1.5194     1.2391     0.2153     -1.0954      4.8607
x1             0.4169     0.0815     5.1142     0.0000      0.2571      0.5767
x2             0.6867     0.1299     5.2860     0.0000      0.4321      0.9413
==============================================================================

Endogenous: x2
Instruments: z2a
Robust Covariance (Heteroskedastic)
Debiased: False
IVResults, id: 0x12a69c8f0

## First-stage test

In [6]:
smf.ols('x2~x1+z2a', data=Ivdata).fit().summary(slim=True)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     x2   R-squared:                       0.295
Model:                            OLS   Adj. R-squared:                  0.281
No. Observations:                 100   F-statistic:                     20.33
Covariance Type:            nonrobust   Prob (F-statistic):           4.22e-08
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      6.3374      0.936      6.769      0.000       4.479       8.196
x1            -0.1319      0.079     -1.669      0.098      -0.289       0.025
z2a           21.8401      4.044      5.401      0.000      13.814      29.866
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
smf.ols('x2~x1+z2a', data=Ivdata).fit(cov_type="HC3").summary(slim=True)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     x2   R-squared:                       0.295
Model:                            OLS   Adj. R-squared:                  0.281
No. Observations:                 100   F-statistic:                     17.75
Covariance Type:                  HC3   Prob (F-statistic):           2.69e-07
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      6.3374      1.061      5.972      0.000       4.258       8.417
x1            -0.1319      0.088     -1.491      0.136      -0.305       0.042
z2a           21.8401      4.389      4.976      0.000      13.238      30.442
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [8]:
stage1a = smf.ols('x2~x1+z2a+z2b', data=Ivdata).fit()
stage1a.f_test('z2a=0,z2b=0')

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=14.539152594855231, p=3.049887117946441e-06, df_denom=96, df_num=2>

## 설명변수 외생성의 검정

In [9]:
Ivdata['v2hat'] = stage1.resid
smf.ols('y~x1+x2+v2hat', data=Ivdata).fit().summary(slim=True)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.722
Model:                            OLS   Adj. R-squared:                  0.714
No. Observations:                 100   F-statistic:                     83.21
Covariance Type:            nonrobust   Prob (F-statistic):           1.33e-26
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.8827      1.284      1.467      0.146      -0.665       4.431
x1             0.4169      0.063      6.591      0.000       0.291       0.543
x2             0.6867      0.131      5.234      0.000       0.426       0.947
v2hat          0.3358      0.150      2.245      0.027       0.039       0.633
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## 도구변수 외생성의 검정

In [10]:
tsls = IV2SLS.from_formula('y~1+x1+[x2~z2a+z2b]', data=Ivdata).fit(cov_type='unadjusted') # see above for ivmodel
tsls.sargan

Sargan's test of overidentification
H0: The model is not overidentified.
Statistic: 0.3355
P-value: 0.5624
Distributed: chi2(1)
WaldTestStatistic, id: 0x12a657f50

linearmodels를 사용하려면 회귀식에 상수항(1)을 명시적으로 표시해 주어야 함에 유의하여야 한다.

## 예제: 교육수익률

In [11]:
import pandas as pd
import statsmodels.formula.api as smf

Schooling = pd.read_csv('csv/Ecdat/Schooling.csv')

fm = 'lwage76~ed76+exp76+smsa76'

### OLS 추정

In [12]:
ols = smf.ols(fm, data=Schooling).fit()
ols.summary(slim=True)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                lwage76   R-squared:                       0.215
Model:                            OLS   Adj. R-squared:                  0.214
No. Observations:                3010   F-statistic:                     274.7
Covariance Type:            nonrobust   Prob (F-statistic):          1.43e-157
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         4.6020      0.063     73.373      0.000       4.479       4.725
smsa76[T.yes]     0.1837      0.016     11.386      0.000       0.152       0.215
ed76              0.0878      0.004     24.610      0.000       0.081       0.095
exp76             0.0411      0.002     17.985      0.000       0.037       0.046
=================================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### IQ를 대리변수로 사용

In [13]:
fm_iq = fm + '+iqscore'
smf.ols(fm_iq, data=Schooling).fit().summary(slim=True)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                lwage76   R-squared:                       0.192
Model:                            OLS   Adj. R-squared:                  0.190
No. Observations:                2061   F-statistic:                     122.0
Covariance Type:            nonrobust   Prob (F-statistic):           1.70e-93
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         4.4761      0.089     50.331      0.000       4.302       4.650
smsa76[T.yes]     0.1548      0.019      8.145      0.000       0.118       0.192
ed76              0.0657      0.005     13.286      0.000       0.056       0.075
exp76             0.0451      0.003     16.288      0.000       0.040       0.051
iqscore           0.0044      0.001      6.972      0.000       0.003       0.006
=================================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.13e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### First-stage regression

In [14]:
fm_stage1 = 'ed76~exp76+smsa76+momed'
fm_stage1

'ed76~exp76+smsa76+momed'

In [15]:
stage1 = smf.ols(fm_stage1, data=Schooling).fit()
stage1.summary(slim=True)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   ed76   R-squared:                       0.488
Model:                            OLS   Adj. R-squared:                  0.487
No. Observations:                3010   F-statistic:                     953.2
Covariance Type:            nonrobust   Prob (F-statistic):               0.00
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        13.9863      0.179     78.008      0.000      13.635      14.338
smsa76[T.yes]     0.4783      0.078      6.110      0.000       0.325       0.632
exp76            -0.3690      0.009    -41.498      0.000      -0.386      -0.352
momed             0.2132      0.012     17.327      0.000       0.189       0.237
=================================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Two stage least squares

In [16]:
from linearmodels import IV2SLS

iv_model = IV2SLS.from_formula('lwage76~1+[ed76~momed]+exp76+smsa76', data=Schooling) # 1+!!!
tsls = iv_model.fit(cov_type='unadjusted')
tsls

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                lwage76   R-squared:                      0.1499
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1490
No. Observations:                3010   F-statistic:                    339.12
Date:                Mon, Jun 24 2024   P-value (F-stat)                0.0000
Time:                        08:43:17   Distribution:                  chi2(3)
Cov. Estimator:            unadjusted                                         
                                                                              
                               Parameter Estimates                               
=================================================================================
               Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
---------------------------------------------------------------------------------
Intercept         3.6712     0.2044     17.960     0.0000      3.2705      4.0718
exp76             0.0644     0.0054     11.925     0.0000      0.0538      0.0750
smsa76[T.yes]     0.1501     0.0182     8.2525     0.0000      0.1144      0.1857
ed76              0.1442     0.0123     11.712     0.0000      0.1201      0.1684
=================================================================================

Endogenous: ed76
Instruments: momed
Unadjusted Covariance (Homoskedastic)
Debiased: False
IVResults, id: 0x12a69cd70

In [17]:
tsls_h = iv_model.fit(cov_type='robust')
tsls_h

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                lwage76   R-squared:                      0.1499
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1490
No. Observations:                3010   F-statistic:                    357.82
Date:                Mon, Jun 24 2024   P-value (F-stat)                0.0000
Time:                        08:43:17   Distribution:                  chi2(3)
Cov. Estimator:                robust                                         
                                                                              
                               Parameter Estimates                               
=================================================================================
               Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
---------------------------------------------------------------------------------
Intercept         3.6712     0.2023     18.144     0.0000      3.2746      4.0677
exp76             0.0644     0.0053     12.056     0.0000      0.0540      0.0749
smsa76[T.yes]     0.1501     0.0180     8.3391     0.0000      0.1148      0.1854
ed76              0.1442     0.0122     11.778     0.0000      0.1202      0.1682
=================================================================================

Endogenous: ed76
Instruments: momed
Robust Covariance (Heteroskedastic)
Debiased: False
IVResults, id: 0x12b0352b0

### Endogeneity test

In [18]:
stage1 = smf.ols(fm_stage1, data=Schooling).fit()
Schooling['vhat'] = stage1.resid
aux = smf.ols(fm + '+vhat', data=Schooling).fit()
aux.summary(slim=True)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                lwage76   R-squared:                       0.222
Model:                            OLS   Adj. R-squared:                  0.221
No. Observations:                3010   F-statistic:                     214.0
Covariance Type:            nonrobust   Prob (F-statistic):          9.56e-162
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         3.6712      0.196     18.754      0.000       3.287       4.055
smsa76[T.yes]     0.1501      0.017      8.618      0.000       0.116       0.184
ed76              0.1442      0.012     12.230      0.000       0.121       0.167
exp76             0.0644      0.005     12.452      0.000       0.054       0.075
vhat             -0.0621      0.012     -5.017      0.000      -0.086      -0.038
=================================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [19]:
smf.ols(fm + '+vhat', data=Schooling).fit(cov_type='HC3').summary(slim=True)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                lwage76   R-squared:                       0.222
Model:                            OLS   Adj. R-squared:                  0.221
No. Observations:                3010   F-statistic:                     217.3
Covariance Type:                  HC3   Prob (F-statistic):          5.44e-164
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         3.6712      0.196     18.732      0.000       3.287       4.055
smsa76[T.yes]     0.1501      0.017      8.717      0.000       0.116       0.184
ed76              0.1442      0.012     12.161      0.000       0.121       0.167
exp76             0.0644      0.005     12.399      0.000       0.054       0.075
vhat             -0.0621      0.013     -4.888      0.000      -0.087      -0.037
=================================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

### Overidentification test

In [20]:
# Don't miss out 1 below
tsls2 = IV2SLS.from_formula('lwage76~1+[ed76~momed+daded]+exp76+smsa76', data=Schooling).fit()
tsls2.sargan

Sargan's test of overidentification
H0: The model is not overidentified.
Statistic: 3.2653
P-value: 0.0708
Distributed: chi2(1)
WaldTestStatistic, id: 0x12b0b9490

### Robust overidentification test

In [21]:
# first stage regression
stage1 = smf.ols('ed76~momed+daded+exp76+smsa76', data=Schooling).fit()
Schooling['ed76hat'] = stage1.fittedvalues
# orthogonalized overidentifying instruments
w1 = IV2SLS.from_formula('daded~1+[ed76hat~ed76]+exp76+smsa76', data=Schooling).fit().resids
# multiply w1 to 2SLS residuals
u = IV2SLS.from_formula('lwage76~1+[ed76~momed+daded]+exp76+smsa76', data=Schooling).fit().resids
Schooling['w1u'] = w1*u
# get the n*R2 stat
Schooling['one'] = [1]*len(Schooling)
aux = smf.ols('one~w1u-1', data=Schooling).fit()
stat = aux.nobs*aux.rsquared
stat

3.185349075973105

In [22]:
# p-value
from scipy.stats import chi2

1-chi2.cdf(stat,1)

0.07430112606532835

## 제곱항

### 도구변수 제곱항을 도구변수로 사용

In [23]:
import pandas as pd

Ivdata = pd.read_csv('csv/loedata/Ivdata.csv')
# Require 1!
ivreg = IV2SLS.from_formula('y~1+x1+[x2+I(x2**2)~z2a+I(z2a**2)]', data=Ivdata).fit(cov_type='unadjusted')
ivreg

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                      y   R-squared:                      0.6619
Estimator:                    IV-2SLS   Adj. R-squared:                 0.6514
No. Observations:                 100   F-statistic:                    40.760
Date:                Mon, Jun 24 2024   P-value (F-stat)                0.0000
Time:                        08:43:17   Distribution:                  chi2(3)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      1.9795     1.5627     1.2668     0.2052     -1.0833      5.0423
x1             0.4188     0.0705     5.9396     0.0000      0.2806      0.5571
x2             0.6159     0.5829     1.0567     0.2907     -0.5265      1.7583
I(x2 ** 2)     0.0066     0.0536     0.1236     0.9017     -0.0985      0.1118
==============================================================================

Endogenous: x2, I(x2 ** 2)
Instruments: z2a, I(z2a ** 2)
Unadjusted Covariance (Homoskedastic)
Debiased: False
IVResults, id: 0x12b0d6a20

### 통제함수

In [24]:
Ivdata['v2hat'] = smf.ols('x2~x1+z2a+z2b', data=Ivdata).fit().resid
smf.ols('y~x1+x2+I(x2**2)+v2hat', data=Ivdata).fit().summary(slim=True)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.723
Model:                            OLS   Adj. R-squared:                  0.712
No. Observations:                 100   F-statistic:                     62.06
Covariance Type:            nonrobust   Prob (F-statistic):           1.12e-25
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.9932      1.305      1.528      0.130      -0.597       4.583
x1             0.4217      0.064      6.632      0.000       0.295       0.548
x2             0.5732      0.210      2.734      0.007       0.157       0.989
I(x2 ** 2)     0.0112      0.015      0.731      0.467      -0.019       0.042
v2hat          0.3219      0.150      2.145      0.035       0.024       0.620
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

위의 표준오차들은 (`v2hat`의 계수가 0인 경우를 제외하면) 잘못되었으므로 수정하여야 한다. 더 이상 진행하지 않는다. 코딩 오류의 위험을 무릅쓰면서 파이썬을 사용해야 할 이유를 찾기 어렵다.